In [1]:
# Simple Object Detection using OpenCV without Machine Learning (ML)
# Approach
# 1. Read image
# 2. Convert image to grayscale
# 3. Increase contrast of image using log transformation

In [2]:
import cv2
import numpy as np


# Image list
# 1. No Tumor
img_no_tumor = ["img/no (1).jpg", "img/no (2).jpg", "img/no (3).jpg", "img/no (4).jpg", "img/no (5).jpg", "img/no (6).jpg", "img/no (7).jpg", "img/no (8).jpg", "img/no (9).jpg", "img/no (10).jpg"]
# 2. Tumor
img_yes_tumor = ["img/gg (1).jpg", "img/gg (2).jpg", "img/gg (3).jpg", "img/gg (4).jpg", "img/m (1).jpg", "img/m (2).jpg", "img/m (3).jpg", "img/m (4).jpg", "img/m (5).jpg", "img/m (6).jpg"]

In [3]:
# ==================================================================================================== Step 1. Read image
df_no_tumor = []
df_yes_tumor = []

for img in img_no_tumor:
    df_no_tumor.append(cv2.imread(img))

for img in img_yes_tumor:
    df_yes_tumor.append(cv2.imread(img))

# Show images
# for i in range(5, 10):
#     cv2.imshow("No Tumor", df_no_tumor[i])
#     cv2.imshow("Yes Tumor", df_yes_tumor[i])
#     cv2.waitKey(0)

In [4]:
# ==================================================================================================== Step 2. Convert image to grayscale
gray_no_tumor = []
gray_yes_tumor = []

for img in df_no_tumor:
    gray_no_tumor.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

for img in df_yes_tumor:
    gray_yes_tumor.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

# Show images
# for i in range(5, 10):
#     cv2.imshow("No Tumor", gray_no_tumor[i])
#     cv2.imshow("Yes Tumor", gray_yes_tumor[i])
#     cv2.waitKey(0)

In [5]:
# ==================================================================================================== Step 3. Cropped to ROI
cropped_no_tumor = []
cropped_yes_tumor = []

# Ref: https://github.com/MohamedAliHabib/Brain-Tumor-Detection/blob/master/Brain%20Tumor%20Detection.ipynb
for img in gray_no_tumor:
    # Thresholding
    threshold = cv2.threshold(img.copy(), 40, 255, cv2.THRESH_OTSU)[1]
    # Find contour
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get the largest contour
    c = max(contours, key=cv2.contourArea)
    # Find extreme points
    extleft = tuple(c[c[:, :, 0].argmin()][0])
    extright = tuple(c[c[:, :, 0].argmax()][0])
    exttop = tuple(c[c[:, :, 1].argmin()][0])
    extbot = tuple(c[c[:, :, 1].argmax()][0])
    # Crop the image
    cropped = img[exttop[1] : extbot[1], extleft[0] : extright[0]]
    cropped_no_tumor.append(cropped)

for img in gray_yes_tumor:
    # Thresholding
    threshold = cv2.threshold(img.copy(), 40, 255, cv2.THRESH_OTSU)[1]
    # Find contour
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get the largest contour
    c = max(contours, key=cv2.contourArea)
    # Find extreme points
    extleft = tuple(c[c[:, :, 0].argmin()][0])
    extright = tuple(c[c[:, :, 0].argmax()][0])
    exttop = tuple(c[c[:, :, 1].argmin()][0])
    extbot = tuple(c[c[:, :, 1].argmax()][0])
    # Crop the image
    cropped = img[exttop[1] : extbot[1], extleft[0] : extright[0]]
    cropped_yes_tumor.append(cropped)

# Show cropped images compare to original images
# for i in range(5, 10):
#     cv2.imshow("Original No Tumor", gray_no_tumor[i])
#     cv2.imshow("No Tumor", cropped_no_tumor[i])
#     cv2.imshow("Original Yes Tumor", gray_yes_tumor[i])
#     cv2.imshow("Yes Tumor", cropped_yes_tumor[i])
#     cv2.waitKey(0)

In [6]:
# ==================================================================================================== Step 4. Thresholding
thres_no_tumor = []
thres_yes_tumor = []

adapThres_no_tumor = []
adapThres_yes_tumor = []


thresh = 100

block_size = 55
c = -15


for img in cropped_no_tumor:
    _, thres = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)
    thres_no_tumor.append(thres)
    adapThres = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)
    adapThres_no_tumor.append(adapThres)


for img in cropped_yes_tumor:
    _, thres = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)
    thres_yes_tumor.append(thres)
    adapThres = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)
    adapThres_yes_tumor.append(adapThres)


# stack images side by side
# for i in range(0, 9):
#     stacked_no_tumor = np.hstack((cropped_no_tumor[i], thres_no_tumor[i], adapThres_no_tumor[i]))
#     stacked_yes_tumor = np.hstack((cropped_yes_tumor[i], thres_yes_tumor[i], adapThres_yes_tumor[i]))
#     cv2.imshow("No Tumor", stacked_no_tumor)
#     cv2.imshow("Yes Tumor", stacked_yes_tumor)
#     cv2.waitKey(0)

In [23]:
# ==================================================================================================== Step 5. Identify ROI
cont_no_tumor = []
contdraw_no_tumor = []
cont_yes_tumor = []
contdraw_yes_tumor = []

bound_no_tumor = []
bound_yes_tumor = []

min_scale = 0.005
max_scale = 0.40

k_size = 3
kernel = np.ones((k_size, k_size), np.uint8)
iteration = 1

# Box ratio factor (w/h, h/w)
rect_factor = 2.2

# Box position factor from edge
box_pos_factor = 0.1

# White pixel ratio of total pixel in an image
white_thres = 0.3

# Minimum white pixel in a bounding box
min_white = 0.25
# Find contour, bounding box, bounding box area, draw bounding box
for img in thres_no_tumor:
    # Check if total white pixel is more than
    if (np.sum(img == 255) / (img.shape[0] * img.shape[1])) > white_thres:
        bound_no_tumor.append(np.zeros_like(img))
        cont_no_tumor.append(np.zeros_like(img))
        continue
    # Dilute and Erode
    eroded = cv2.erode(img.copy(), kernel, iterations=iteration)
    dilated = cv2.dilate(eroded, kernel, iterations=iteration)

    # Contour
    # contours, _ = cv2.findContours(eroded, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours, _ = cv2.findContours(dilated, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    new_canvas = np.zeros_like(img)
    cv2.drawContours(new_canvas, contours, -1, (255, 255, 255), 1)
    cont_no_tumor.append(new_canvas)

    # Bounding box
    new_canvas = np.zeros_like(img)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        # Filter By Area, box w and h ratio (cannot be too rectangle), box position (cannot be too close to edge), box are and white pixel ratio
        min_area = min_scale * img.shape[0] * img.shape[1]
        max_area = max_scale * img.shape[0] * img.shape[1]
        area = w * h
        bound_center = (x + w / 2, y + h / 2)
        low_x = box_pos_factor * img.shape[1]
        low_y = box_pos_factor * img.shape[0]
        up_x = img.shape[1] - box_pos_factor * img.shape[1]
        up_y = img.shape[0] - box_pos_factor * img.shape[0]
        if min_area < area < max_area and w / h < rect_factor and h / w < rect_factor and (up_x > bound_center[0] > low_x) and (up_y > bound_center[1] > low_y) and (np.sum(img[y : y + h, x : x + w] == 255) / area) > min_white:
            cv2.rectangle(new_canvas, (x, y), (x + w, y + h), (255, 255, 255), 1)
    bound_no_tumor.append(new_canvas)

for img in thres_yes_tumor:
    # Check if total white pixel is more than
    if (np.sum(img == 255) / (img.shape[0] * img.shape[1])) > white_thres:
        bound_yes_tumor.append(np.zeros_like(img))
        cont_yes_tumor.append(np.zeros_like(img))
        continue
    # Dilute and Erode
    eroded = cv2.erode(img.copy(), kernel, iterations=iteration)
    dilated = cv2.dilate(eroded, kernel, iterations=iteration)

    # Contour
    contours, _ = cv2.findContours(dilated, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    new_canvas = np.zeros_like(img)
    cv2.drawContours(new_canvas, contours, -1, (255, 255, 255), 1)
    cont_yes_tumor.append(new_canvas)

    # Bounding box
    new_canvas = np.zeros_like(img)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        # Filter By Area, box w and h ratio (cannot be too rectangle), box position (cannot be too close to edge)
        min_area = min_scale * img.shape[0] * img.shape[1]
        max_area = max_scale * img.shape[0] * img.shape[1]
        area = w * h
        bound_center = (x + w / 2, y + h / 2)
        low_x = box_pos_factor * img.shape[1]
        low_y = box_pos_factor * img.shape[0]
        up_x = img.shape[1] - box_pos_factor * img.shape[1]
        up_y = img.shape[0] - box_pos_factor * img.shape[0]
        if min_area < area < max_area and w / h < rect_factor and h / w < rect_factor and (up_x > bound_center[0] > low_x) and (up_y > bound_center[1] > low_y) and (np.sum(img[y : y + h, x : x + w] == 255) / area) > min_white:
            cv2.rectangle(new_canvas, (x, y), (x + w, y + h), (255, 255, 255), 1)
    bound_yes_tumor.append(new_canvas)

# Show images
for i in range(0, len(img_no_tumor)):
    stacked_no_tumor = np.hstack((cropped_no_tumor[i], thres_no_tumor[i]))
    stacked_yes_tumor = np.hstack((cropped_yes_tumor[i], thres_yes_tumor[i]))
    stacked_no_tumor_cont_bound = np.hstack((cont_no_tumor[i], bound_no_tumor[i]))
    stacked_yes_tumor_cont_bound = np.hstack((cont_yes_tumor[i], bound_yes_tumor[i]))
    cv2.imshow("No Tumor", stacked_no_tumor)
    cv2.imshow("No Tumor Contour Bound", stacked_no_tumor_cont_bound)
    cv2.imshow("Yes Tumor", stacked_yes_tumor)
    cv2.imshow("Yes Tumor Contour Bound", stacked_yes_tumor_cont_bound)
    cv2.waitKey(0)

cv2.destroyAllWindows()